In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')
df.head(3)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().T.head(2)

In [ ]:
pd.merge(df.describe().T, 
         pd.DataFrame(df.dtypes, columns=['data type']),
         how='right',
         left_index=True, 
         right_index=True).applymap(lambda x: '-' if pd.isnull(x) else x)

In [ ]:
plt.figure(figsize=(10, 6))
sns.displot(df['price'])

In [ ]:
sns.countplot(df['bedrooms']);

In [ ]:
plt.rcParams.update({'font.size': 13})
df.corr()['price'].sort_values(key=lambda x: abs(x), ascending=False).plot.bar(figsize=(10, 3));

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot(x='sqft_living', y='price', marker='o', data=df);

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='bedrooms', y='price', data=df);

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='long', y='lat', hue='price', data=df);

In [ ]:
non_top_1_pct = df[df['price'] >= df['price'].quantile(q=0.01)]
non_top_1_pct.shape, df.shape

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='long', y='lat', hue='price', data=non_top_1_pct,
                edgecolor=None, palette='RdYlGn', alpha=0.2);

In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['date']

In [ ]:
df['year'] = df['date'].apply(lambda x: x.year)
df['month'] = df['date'].apply(lambda x: x.month)
df.head().T

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='month', y='price', data=df);

In [ ]:
# visually inspect the correlation between house pricing and the sales month of the year
df.groupby('month')['price'].mean().plot();

In [ ]:
df.drop('date', axis=1, inplace=True)
df.columns

In [ ]:
df['zipcode'].value_counts().shape[0], df.shape[0]

In [ ]:
# Drop the zipcode column for simplicity
df.drop('zipcode', axis=1, inplace=True)

In [ ]:
df['yr_renovated'].value_counts()

Although there are many `0` values in the yr_renovated column that implies that the house is not renovated, because year renovated has a positive correlation with the price, and `0` is the least possible value, we are going to keep data as is

There is a similar situation with `sqft_basement`

In [ ]:
df['sqft_basement'].value_counts()

Although sqft_basement with `0`  means no basement that is actually a categorical feature, we are going to keep the data as is, considering that the data is positively correlated with `price`

In [ ]:
X = df.drop('price', axis=1).values
y = df['price'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(19, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(19, activation='relu'))

model.add(Dense(1))

model.compile(optimizer='adam',
              loss='mse')

In [ ]:
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          batch_size=128,
          epochs=100,
          verbose=0)

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score

In [ ]:
predictions = model.predict(X_test)

In [ ]:
root_mse = np.sqrt(mean_squared_error(y_test, predictions))
root_mse

In [ ]:
mae = mean_absolute_error(y_test, predictions)
mae

In [ ]:
# How much variance is bein explained by the model
explained_variance_score(y_test, predictions)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions)
plt.plot(y_test, y_test, 'r')